In [1]:
# the notebook will reload external python modules;
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../codes')
import os
import cv2
from yolov3.darknet import *

In [3]:
yolov3_path = '../codes/yolov3/'

In [4]:
blocks = parse_cfg(os.path.join(yolov3_path, 'cfg/yolov3.cfg'))

In [5]:
print(create_modules(blocks))

({'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (batch_n

/home/wesley/miniconda3/envs/mosquitoes/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


## Testing Forward Pass

In [6]:
def get_test_input(input_path):
    img = cv2.imread(input_path)
    img = cv2.resize(img, (416,416))          #Resize to the input dimension
    img_ =  img[:,:,::-1].transpose((2,0,1))  # BGR -> RGB | H X W C -> C X H X W 
    img_ = img_[np.newaxis,:,:,:]/255.0       #Add a channel at 0 (for batch) | Normalise
    img_ = torch.from_numpy(img_).float()     #Convert to float
    img_ = Variable(img_)                     # Convert to Variable
    return img_

In [10]:
model = Darknet(os.path.join(yolov3_path,'cfg/yolov3.cfg'))
inp = get_test_input(os.path.join(yolov3_path, 'dog-cycle-car.png'))
pred = model(inp, torch.cuda.is_available())
print(pred)
print(pred.shape)

/home/wesley/miniconda3/envs/mosquitoes/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))
/home/wesley/miniconda3/envs/mosquitoes/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/home/wesley/miniconda3/envs/mosquitoes/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


tensor([[[ 14.7504,  15.2918, 123.3044,  ...,   0.4847,   0.5598,   0.4767],
         [ 16.7632,  15.9272, 126.4591,  ...,   0.5073,   0.5352,   0.4487],
         [ 15.7479,  13.3667, 360.6731,  ...,   0.5719,   0.5121,   0.5494],
         ...,
         [412.3139, 412.3129,  10.7758,  ...,   0.5696,   0.4874,   0.5258],
         [412.3039, 411.5764,  13.2512,  ...,   0.4810,   0.4981,   0.5168],
         [412.1143, 411.4541,  38.1084,  ...,   0.5268,   0.4725,   0.5214]]])
torch.Size([1, 10647, 85])


## Load weights

In [8]:
model.load_weights(os.path.join(yolov3_path,'yolov3.weights'))